---
title: "My Document"
format: html
bibliography: references.bib
link-citations: true
---


# Retention Time Alignment

Due to systemic error, datasets of chromatogams of samples run under the same experimental conditions often will exhibit retention time shifting for the same compound peak. Inter-sample data analysis requires that features are aligned within the same vector. The magnitude and direction of run on run peak shift is unique for each peak for each run, within a distribution, creating a complex problem. A traditional approach is to reduce the dimensionality of the chromatograms through an aggregate measure such as peak area, discarding the time axis in favor of an element-wise ordering. This approach has two downsides - the first is the manual marking, grouping and ordering of peaks across the sampleset, which is subjective and often irreproducible [@zheng_2017], the second is the arbitrary loss of information fed to the statistical model, specifically the signal shapes [@nielsen_1998]. @bos-RecentApplicationsChemometrics-2020 lists correlation-optimized warping (COW), dynamic time warping (DTW), and correlation-optimized shifting (COSHIFT) as the most popular methods of alignment.

## DTW

Dynamic time warping is a method of aligning two time series, a reference series (to be aligned to) and a query series (to be aligned), originally developed in the context of speech recognition technology [@velichko_1970]. Alignment achieved by performing localizaed warping in the form of  stretching and compressing of the query series until a pre-defined level of alignment is reached, measured as the minimization of the distances between the two series. The distance between the two signals is measured as the sum of the distances between each series elementwise pairing [@giorgino_2009]. [@jiao_2015] has shown that DTW is an appropriate method of aligning organic sample chromatogram datasets. [@bork_2013] has discussed the importance of DTW for process monitoring. They describe how traditional DTW can alter y-axis values while aligning the x, however an extension to the method termed 'Derivative Dynamic Time Warping' [@keogh_2001] respects the shape of each series by observing their first derivatives, reducing unnecessary modifications.

### Outputting Aligned Tensors Through DTW

Tomasi discusses signal alignment through DTW in [@tomasi_2004, p. 7, sec. 2.3.3.] where they note that DTW does not itself produce aligned series of the same length, rather outputting shortened or lengthened series depending on the warping path taken. For stacking of sample signals into a tensor, the signals need to be the same length. This is not the intent of the design of the DTW algorithm, which rather is used to output the cost of aligning the series in the form of a distance metric. They do state that a desired synchronization can be achieved by either taking the mean value of intervals of stretching in the query (removing repeated time points in the warping), or an asymmetric warping algorithm which directly maps the query to the reference, but this can cause discontinuities in the warped signal, quoting @kassidas_1997.

## COW

@skov_2006 discussed the use of the COW algorithm for alignment of chromatographic data. The Correlation Optimized Warping (COW) algorithm was developed by Nielsen et al. [@nielsen_1998] for the purpose of data prepraration for multivariate statistical analysis.

COW can be used on 2d and 3d data and the output can be fed directly into models such as PCA.[@nielsen_1998].

COW is similar to DTW constrained to a number of windows along the time axis [@nielsen_1998].

What is COW?

COW aligns one signal onto another through localized linear stretching and compression of its time axis.
COW was developed by Nielsen et al. who demonstrated its use on single and multi-channel HPLC-DAD chromatograms of fungal extracts [@nielsen_1998].

The theory of COW is as follows (using notation from [@nielsen_1998]): for two signals, the 'target' (T) and the 'profile' (P), the two signals are divided up into a finite number of sections ($N=\frac{L_p}{m}$), each of which is internally warped to maximise alignment, resulting in an aligned signal (P'). This operation is constrained to ensure that time ordering is retained. Within each section, warping can either stretch or compress the sections, and in the case of a length mismatch, P' is linearly interpolated to match the length of T. The warping magnitude is constrained through parameter $t$, called 'the slack'. For P and T of different length, slack is defined as lying within the range $\Delta \pm t$ where $\Delta=\frac{L_T}{N}-m$.  Warping is performed section by section, and for the correlation coefficient of each pair is calculated and the end-point of the process is the maximisation of the sum of correlation coefficients. The identification of the section warping optimization is identified through dynamic programming. The calculation only requires specification of the segment length and slack parameters. Nielsen et al. demonstrated that COW performed better when fitting 3D data than 2D as the 'spectral information' restricted overfitting. In their example, they showed that subsetting the dataset to the target interval and baseline correction improved alignment. They also recommended using the cubed correlation coefficient over the base form as it selects for optimzied alignment without compromise. [@nielsen_1998].

DTW and COW were compared by Tomasi et al. [@tomasi_2004]. The MATLAB code the group wrote for COW can be found on the website of [Chemometrics Group of Copenhagen](https://ucphchemometrics.com/warping/).

COW has been adapted for 2D chromtaography by several indepedant research groups [@zhang_2008, @gros_2012].


## DTW Algorithm

notes from @giorgino_ComputingVisualizingDynamic_2009.

### Introduction

$X$: the *test* or *query*
$Y$: the *reference*

*i*: index of $X$
*j*: index of $Y$

*f*: *local dissimilarity function* defined between pairs of $x_i$ and $y_j$. Non negative: $$d(i, j)=f(x_i, y_j) \geq 0$$

$d$: cross-distance matrix between $X$ and $Y$

$\phi(k)$: warping curve $$\phi(k)=(\phi_x(k), \phi_y(k))$$

Where $\phi_x(k), \phi_y(k)$ outputs an integer from 1 to N.

$\phi_x$ remaps $X$ time indices
$\phi_y$ remaps $Y$ time indices

There is a average accumulated distortion between the warped $X$ and $Y$: $$d_\phi(X, Y) = \sum_{k=1}^{T} d(\phi_x(k), \phi_y(k)) \frac{m_\phi(k)}{M_\phi}$$

$m_\phi(k)$: per-step weighting coefficient, 
$M_\phi(k)$: normalizing constant of $m_\phi(k)$

 $\phi$ is constrained to ensure reasonable results. One constraint is monotonicity to ensure time ordering/avoid unnecessary loops: $$\phi_x(k+1) \geq \phi_x(k)$$  $$\phi_y(k+1) \geq \phi_y(k)$$

The goal of DTW is to minimize the distance between $X$ and $Y$: $$D(X, Y)=min \space d_\phi(X, Y)$$

Note: this mentions that Y is also deformed "The deformation of the time axes of $X$ **and** $Y$" (emphasis mine).

DTW can be computed in $O(N \cdot M)$ tiime.

$D(X, Y)$: "minimum global dissimilarity", "DTW distance". Stretch insensitive measure of the 'inherent difference' between $X$ and $Y$


## dtwalign

`dtwalign` is a Python package that includes outputting the alignment path


In [ ]:
from dtwalign import dtw
import pandas as pd
import numpy as np
from wine_analysis_hplc_uv import definitions
import seaborn as sns
import matplotlib.pyplot as plt
from .dtw_methods import DTWNotebookMethods
from wine_analysis_hplc_uv.signal_processing.mindex_signal_processing import (
    SignalProcessor,
)

scipro = SignalProcessor()
df = pd.read_parquet(definitions.XPRO_YPRO_DOWNSAMPLED_PARQ_PATH)
df.head()

In [ ]:
x = df["176"].pipe(lambda df: df.set_axis(["154-query"], axis=1))
y = df["177"].pipe(lambda df: df.set_axis(["177-ref"], axis=1))

In [ ]:
def align(x, y, kwargs={}):
    res = dtw(x=x, y=y, **kwargs)

    display(res.plot_path())

    fig, ax = plt.subplots(1)
    aligned_x = x.iloc[res.get_warping_path()].rename(
        {"154-query": "154-aligned_query"}, axis=1
    )
    aligned_x.plot(ax=ax)
    y.plot(ax=ax)
    # y.iloc[res.get_warping_path('reference')].plot(ax=ax)
    # x.plot(ax=ax)
    return aligned_x


align_x = align(x, y)

In [ ]:
# reindex post-alignment to set index as monotonic increasing rather than repeating indices

align_x = (align_x
        .set_index(
        pd.timedelta_range(start=df.index[0], end=df.index[-1], freq="2S")
    ))


align_x.plot()

In [ ]:
fig, ax = plt.subplots(1)
x.plot(ax=ax)
align_x.plot(ax=ax)

As we can see, numerous peaks are lost post-alignment. But what about a localized warping approach?


In [ ]:
align(x, y, kwargs=dict(window_size=100))

In [ ]:
fig, ax = plt.subplots(1)

x.plot(ax=ax)
y.plot(ax=ax)

In [ ]:

aligned_xx = align(x, y, kwargs=dict(window_type="sakoechiba", window_size=10))

fig, ax = plt.subplots(1)
aligned_xx.plot(ax=ax)
x.plot(ax=ax)
plt.tight_layout()

Success! Selecting a Sakoechiba window with size 10 dramaticaly alters the behavior of the warping, preserving peaks that were otherwise lost. It also appears as though the (in the default setting at least), baseline height differences affect the warp. It appears that reducing the distance/cost pre-warp will reduce overall warping errors.

Do do a grid search (so to speak) I need to iterate through a list of keywords. You can assign *N* columns during one groupby operation, so I could iterate inside a pipe, say with a list comprehension..

What are the parameters to iterate through? To start, do a Sakoechiba window with window size 1, 10, 30, 60, 100.

In [ ]:
kwarg_list = [
    dict(window_type='sakoechiba',window_size=1),
    dict(window_type='sakoechiba',window_size=10),
    dict(window_type='sakoechiba',window_size=30),
    dict(window_type='sakoechiba',window_size=60),
    dict(window_type='sakoechiba',window_size=100),
 ]

## Windowing

When aligning a query and reference chromatogram of similar samples we have the expectation that peaks within $d_m$ distance, where $d_m$ is misalignment distance, are the same compound and will be aligned to the same retention time post warping. Conversely, we do not expect regions where the query has peaks but the reference does not to be altered. Unfortunatey, the algorithm does not by default contain that information, and without a global constraint it will drastically alter the query along the entire serie to minimize what it percieves as the distance between the two, including compressing peaks present in the query but not the reference:

As we cam see the query is modified by the algorithm to match the reference in both the x and y axes. This leads to an unexpectedly deformed query. What is needed is to restrict warping to localized regions in the form of a windowing operation with windows of specific geometry. One such is the Sakoe-Chiba band [@sakoe_1978] which restricts how far apart two elements can be when matched: $$|\phi_x(k)-\phi_y(k)| \leq T_0$$ where $T_0$ is the absolute time deviation between two matched elements, specified by the user. As described by @giorgino_ComputingVisualizingDynamic_2009, this creates a boundary within the alignment plane within which the warping path can exist. For a window of size 10:

In [ ]:
# get the windowed aligned x, assign a regular frequency time index, rename axes
def sakoechiba_window_10_warp(x, y):
    sakoechiba_10_x_align = (x
    .iloc[dtw(x,y, window_type="sakoechiba", window_size=10).get_warping_path()]
    .pipe(
        lambda df: df
        .set_index(
            pd.timedelta_range(start=df.index[0], end=df.index[-1], freq="2S")
        )
        .rename_axis('mins')
        )
    .pipe(lambda df: df.set_axis(pd.MultiIndex.from_arrays([['154'],['aligned'],[10], ['abs']],names =['sample','status','window_size','unit']),axis=1, )
        )
    )
    return sakoechiba_10_x_align

sakoechiba_10_x_align = sakoechiba_window_10_warp(x, y)
sakoechiba_10_x_align.plot()

Looks promising, how does it compare to the query and the reference?

In [ ]:
# join x, y and aligned x on index

x = x.pipe(lambda df: df.set_axis(axis=1, labels=pd.MultiIndex.from_arrays([['154'],['query'],['NA'],['abs']], names =['sample','status','window_size','unit'])))
y = y.pipe(lambda df: df.set_axis(axis=1, labels=pd.MultiIndex.from_arrays([['177'],['ref'],['NA'],['abs']], names =['sample','status','window_size','unit'])))

In [ ]:
nb_mtds = DTWNotebookMethods()

plot_data, g = nb_mtds.query_ref_align_plot(x=x, y=y, x_align=sakoechiba_10_x_align)

Now let's compare that to an alignment without a global constraint:

In [ ]:
def plot_unconstrained_warp(x, y):

    x_align = (x
    .iloc[dtw(x,y).get_warping_path()]
    .pipe(
        lambda df: df
        .set_index(
            pd.timedelta_range(start=df.index[0], end=df.index[-1], freq="2S")
        )
        .rename_axis('mins')
        )
    .pipe(lambda df: df.set_axis(pd.MultiIndex.from_arrays([['154'],['aligned'],[10], ['abs']],names =['sample','status','window_size','unit']),axis=1, )
        )
    )

    plot_data, g = query_ref_align_plot(x=x, y=y, x_align=x_align)
    
    return None
    
plot_unconstrained_warp(x, y)
